In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

from typing import List,Dict
from elmo_on_md.data_loaders.sentiment_loader import SentimentLoader
from elmo_on_md.evaluation.sentiment_analysis import SentimentAnalysis
from elmo_on_md.evaluation.model_loader import load_model
from ELMoForManyLangs.elmoformanylangs import Embedder

import torch
torch.device('cpu')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline


In [4]:
loader = SentimentLoader()
sentiment_data = loader.load_data()


In [5]:
elmo = load_model('original')
other_elmo = load_model('pos_weight8_lr-4_new_tags_30epochs')

2019-08-29 22:48:45,895 INFO: char embedding size: 2289
2019-08-29 22:48:46,917 INFO: word embedding size: 189561
2019-08-29 22:48:57,695 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(189561, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2289, 50, padding_idx=2286)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

In [6]:


sentiment = SentimentAnalysis([elmo,other_elmo],lr=1e-4)
sentences = sentiment_data['train']['sentences']
labels = sentiment_data['train']['labels']

tokens_train,tokens_test, labels_train,labels_test = train_test_split(sentences, labels, test_size=0.2, random_state=1)

train_set = {'sentences':tokens_train,'labels':labels_train}
validate_set = {'sentences':tokens_test,'labels':labels_test}

sentiment.train(train_set,validate_set,n_epochs=200)



2019-08-29 22:49:32,269 INFO: 129 batches, avg len: 22.3


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 2.00 GiB total capacity; 1.18 GiB already allocated; 492.80 KiB free; 110.84 MiB cached)

In [ ]:
sentences = sentiment_data['train']['sentences']
labels = sentiment_data['train']['labels']
sentiment_data1 = dict()
sentiment_data2 = dict()
sentiment_data1['sentences']=sentences[:len(sentences)//2]
sentiment_data2['sentences']=sentences[len(sentences)//2:]
sentiment_data1['labels']=labels[:len(sentences)//2]
sentiment_data2['labels']=labels[len(sentences)//2:]




In [ ]:
train_preds1 = sentiment.predict(sentiment_data1)
train_preds2 = sentiment.predict(sentiment_data2)

In [7]:

np.concatenate([train_preds1,train_preds2])
print(confusion_matrix(labels,np.concatenate([train_preds1,train_preds2])))
test_preds = sentiment.predict(validate_set)
print(confusion_matrix(validate_set['labels'],test_preds))
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.metrics import precision_recall_fscore_support,classification_report
print(classification_report(sentiment_data['train']['labels'],np.concatenate([train_preds1,train_preds2]),target_names=['bad','neutral','good']))
print(classification_report(validate_set['labels'],test_preds,target_names=['bad','neutral','good']))

NameError: name 'train_preds1' is not defined

In [6]:

np.concatenate([train_preds1,train_preds2])
print(confusion_matrix(labels,np.concatenate([train_preds1,train_preds2])))
test_preds = sentiment.predict(validate_set)
print(confusion_matrix(validate_set['labels'],test_preds))
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.metrics import precision_recall_fscore_support,classification_report



[[6358  421   35]
 [ 189 2926   17]
 [  17   33  248]]


I0829 00:57:28.219690 15848 elmo.py:97] 33 batches, avg len: 22.6
I0829 00:58:30.079703 15848 elmo.py:211] Finished 1000 sentences.
I0829 00:59:57.356936 15848 elmo.py:211] Finished 2000 sentences.


[[1196  139   10]
 [  77  556    8]
 [   9   18   36]]


In [7]:
print(classification_report(sentiment_data['train']['labels'],np.concatenate([train_preds1,train_preds2]),target_names=['bad','neutral','good']))
print(classification_report(validate_set['labels'],test_preds,target_names=['bad','neutral','good']))

              precision    recall  f1-score   support

         bad       0.97      0.93      0.95      6814
     neutral       0.87      0.93      0.90      3132
        good       0.83      0.83      0.83       298

    accuracy                           0.93     10244
   macro avg       0.89      0.90      0.89     10244
weighted avg       0.93      0.93      0.93     10244

              precision    recall  f1-score   support

         bad       0.93      0.89      0.91      1345
     neutral       0.78      0.87      0.82       641
        good       0.67      0.57      0.62        63

    accuracy                           0.87      2049
   macro avg       0.79      0.78      0.78      2049
weighted avg       0.88      0.87      0.87      2049



In [8]:

test_preds = sentiment.predict(sentiment_data['test'])
print(confusion_matrix(sentiment_data['test']['labels'],test_preds))

print(classification_report(sentiment_data['test']['labels'],test_preds,target_names=['bad','neutral','good']))

I0829 07:37:26.145551 15848 elmo.py:97] 40 batches, avg len: 22.7
I0829 07:39:17.490673 15848 elmo.py:211] Finished 1000 sentences.
I0829 07:40:27.807735 15848 elmo.py:211] Finished 2000 sentences.


[[1509  170   19]
 [  93  686   11]
 [  17   26   29]]
              precision    recall  f1-score   support

         bad       0.93      0.89      0.91      1698
     neutral       0.78      0.87      0.82       790
        good       0.49      0.40      0.44        72

    accuracy                           0.87      2560
   macro avg       0.73      0.72      0.72      2560
weighted avg       0.87      0.87      0.87      2560



In [7]:
print(classification_report(sentiment_data['train']['labels'],np.concatenate([train_preds1,train_preds2]),target_names=['bad','neutral','good']))
print(classification_report(sentiment_data['test']['labels'],test_preds,target_names=['bad','neutral','good']))

              precision    recall  f1-score   support

         bad       0.97      0.93      0.95      6814
     neutral       0.86      0.95      0.90      3132
        good       0.85      0.85      0.85       298

    accuracy                           0.93     10244
   macro avg       0.89      0.91      0.90     10244
weighted avg       0.94      0.93      0.93     10244

              precision    recall  f1-score   support

         bad       0.94      0.88      0.91      1698
     neutral       0.77      0.87      0.81       790
        good       0.48      0.42      0.44        72

    accuracy                           0.87      2560
   macro avg       0.73      0.72      0.72      2560
weighted avg       0.87      0.87      0.87      2560



In [4]:
train_preds = sentiment.predict(sentiment_data['train'])
print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
test_preds = sentiment.predict(sentiment_data['test'])
print(confusion_matrix(sentiment_data['test']['labels'],test_preds))
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.metrics import precision_recall_fscore_support,classification_report
print(classification_report(sentiment_data['train']['labels'],train_preds,target_names=['bad','neutral','good']))
print(classification_report(sentiment_data['test']['labels'],test_preds,target_names=['bad','neutral','good']))


I0826 00:16:09.515891 16988 elmo.py:97] 161 batches, avg len: 17.0
I0826 00:17:18.708282 16988 elmo.py:211] Finished 1000 sentences.
I0826 00:18:09.752365 16988 elmo.py:211] Finished 2000 sentences.
I0826 00:19:11.690146 16988 elmo.py:211] Finished 3000 sentences.
I0826 00:20:04.656741 16988 elmo.py:211] Finished 4000 sentences.
I0826 00:21:08.080207 16988 elmo.py:211] Finished 5000 sentences.
I0826 00:22:00.888988 16988 elmo.py:211] Finished 6000 sentences.
I0826 00:22:59.192112 16988 elmo.py:211] Finished 7000 sentences.
I0826 00:23:49.285412 16988 elmo.py:211] Finished 8000 sentences.
I0826 00:24:28.830652 16988 elmo.py:211] Finished 9000 sentences.
I0826 00:25:37.947032 16988 elmo.py:211] Finished 10000 sentences.


[[6336  357  121]
 [ 223 2841   68]
 [  18   14  266]]


I0826 00:26:17.425742 16988 elmo.py:97] 40 batches, avg len: 17.1
I0826 00:27:16.036770 16988 elmo.py:211] Finished 1000 sentences.
I0826 00:28:21.057963 16988 elmo.py:211] Finished 2000 sentences.


[[1521  144   33]
 [  98  658   34]
 [  16   20   36]]
              precision    recall  f1-score   support

         bad       0.96      0.93      0.95      6814
     neutral       0.88      0.91      0.90      3132
        good       0.58      0.89      0.71       298

    accuracy                           0.92     10244
   macro avg       0.81      0.91      0.85     10244
weighted avg       0.93      0.92      0.92     10244

              precision    recall  f1-score   support

         bad       0.93      0.90      0.91      1698
     neutral       0.80      0.83      0.82       790
        good       0.35      0.50      0.41        72

    accuracy                           0.87      2560
   macro avg       0.69      0.74      0.71      2560
weighted avg       0.87      0.87      0.87      2560



In [4]:
train_preds = sentiment.predict(sentiment_data['train'])
print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
test_preds = sentiment.predict(sentiment_data['test'])
print(confusion_matrix(sentiment_data['test']['labels'],test_preds))
from sklearn.metrics import precision_recall_fscore_support,classification_report
from sklearn.metrics import precision_recall_fscore_support,classification_report
print(classification_report(sentiment_data['train']['labels'],train_preds,target_names=['bad','neutral','good']))
print(classification_report(sentiment_data['test']['labels'],test_preds,target_names=['bad','neutral','good']))


I0824 12:49:25.633167 21040 elmo.py:97] 161 batches, avg len: 17.0
I0824 12:50:33.290285 21040 elmo.py:211] Finished 1000 sentences.
I0824 12:51:23.120063 21040 elmo.py:211] Finished 2000 sentences.
I0824 12:52:18.743351 21040 elmo.py:211] Finished 3000 sentences.
I0824 12:53:30.671048 21040 elmo.py:211] Finished 4000 sentences.
I0824 12:54:28.752764 21040 elmo.py:211] Finished 5000 sentences.
I0824 12:55:16.594856 21040 elmo.py:211] Finished 6000 sentences.
I0824 12:56:22.606369 21040 elmo.py:211] Finished 7000 sentences.
I0824 12:57:20.747925 21040 elmo.py:211] Finished 8000 sentences.
I0824 12:58:08.270870 21040 elmo.py:211] Finished 9000 sentences.
I0824 12:58:56.832039 21040 elmo.py:211] Finished 10000 sentences.
..\elmo_on_md\evaluation\sentiment_analysis.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return output


[[5886  872   56]
 [ 182 2918   32]
 [  15   45  238]]


I0824 12:59:34.101390 21040 elmo.py:97] 40 batches, avg len: 17.1
I0824 13:00:28.782982 21040 elmo.py:211] Finished 1000 sentences.
I0824 13:01:30.980684 21040 elmo.py:211] Finished 2000 sentences.


[[1389  287   22]
 [  75  703   12]
 [  11   32   29]]
              precision    recall  f1-score   support

         bad       0.97      0.86      0.91      6814
     neutral       0.76      0.93      0.84      3132
        good       0.73      0.80      0.76       298

    accuracy                           0.88     10244
   macro avg       0.82      0.86      0.84     10244
weighted avg       0.90      0.88      0.89     10244

              precision    recall  f1-score   support

         bad       0.94      0.82      0.88      1698
     neutral       0.69      0.89      0.78       790
        good       0.46      0.40      0.43        72

    accuracy                           0.83      2560
   macro avg       0.70      0.70      0.69      2560
weighted avg       0.85      0.83      0.83      2560



In [1]:
from sklearn.metrics import precision_recall_fscore_support,classification_report
print(classification_report(sentiment_data['train']['labels'],train_preds,target_names=['bad','neutral','good']))

NameError: name 'sentiment_data' is not defined

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,classification_report
print(classification_report(sentiment_data['test']['labels'],test_preds,target_names=['bad','neutral','good']))

In [ ]:
sentiment.train(train_set,validate_set,n_epochs=100)

In [ ]:
train_preds = sentiment.predict(sentiment_data['train'])
print(confusion_matrix(sentiment_data['train']['labels'],train_preds))
test_preds = sentiment.predict(sentiment_data['test'])
print(confusion_matrix(sentiment_data['test']['labels'],test_preds))

In [6]:
import numpy as np
from tensorflow.python.summary.event_accumulator import EventAccumulator


log_file = "D://Projects/events.out.tfevents.1566034114.DESKTOP-OE11P6R.5500.0"
plot_tensorflow_log(log_file)

ModuleNotFoundError: No module named 'tensorflow.python.summary.event_accumulator'